# 💬 Chat with Unsloth GPT-OSS (Streaming in a Notebook)

This notebook demonstrates a **minimal chat loop** using the **Unsloth** `FastLanguageModel` with **streaming generation** via Hugging Face’s `TextIteratorStreamer`.

It supports both **full-precision** and **4-bit quantized** models, and shows how to set the `reasoning_effort` parameter in the chat template.

---

## ✅ Overview

**You’ll learn how to:**
- Load and prepare an Unsloth model & tokenizer  
- Stream responses token-by-token  
- Control sampling (temperature, top-p, etc.)  
- Run an interactive REPL chat loop  
- Adjust reasoning effort (`"low"`, `"medium"`, `"high"`)

---

## 🧰 Requirements

**Install dependencies:

```bash
pip install unsloth transformers accelerate
# For 4-bit models:
pip install bitsandbytes




## ✅ Recommended Environment Note

**Note:**  
> For interactive sessions that use the Python `input()` function,  
> please run this notebook in a **web browser (Jupyter Notebook or JupyterLab)** `using jupyter lab --no-browser --port 8888` command.  
> The **VS Code Jupyter extension** currently does not fully support interactive `input()` prompts,  
> which may cause the session to hang or fail to capture user input correctly.


## 🧠 Model Options

**You can use either full precision or 4-bit quantized models:

| Type         | Model Name                                                                      |
| ------------ | ------------------------------------------------------------------------------- |
| Full / MXFP4 | `unsloth/gpt-oss-20b`, `unsloth/gpt-oss-120b`                                   |
| 4-bit (bnb)  | `unsloth/gpt-oss-20b-unsloth-bnb-4bit`, `unsloth/gpt-oss-120b-unsloth-bnb-4bit` |

To enable 4-bit quantization, set:

load_in_4bit = True



## ⚙️ Key Parameters

| Parameter                | Description                       |
| ------------------------ | --------------------------------- |
| `max_seq_length=4096`    | Context length for the model      |
| `temperature=0.7`        | Sampling temperature (creativity) |
| `top_p=0.9`              | Nucleus sampling probability      |
| `reasoning_effort="low"` | Adjusts reasoning complexity      |
| `max_new_tokens=256`     | Number of tokens to generate      |
| `do_sample=True`         | Enables stochastic generation     |


## 1️⃣ Load the model

In [1]:
import os, torch, threading
from unsloth import FastLanguageModel
from transformers import TextIteratorStreamer


# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit", # 20B model using bitsandbytes 4bit quantization
    "unsloth/gpt-oss-120b-unsloth-bnb-4bit",
    "unsloth/gpt-oss-20b", # 20B model using MXFP4 format
    "unsloth/gpt-oss-120b",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b-unsloth-bnb-4bit",
    dtype = None, # None for auto detection
    max_seq_length = 4096, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)


FastLanguageModel.for_inference(model)
device = next(model.parameters()).device
tokenizer.pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id




🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.11: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## 2️⃣ Initialize conversation

In [2]:
messages = [
    {"role": "system", "content": "You are a helpful assistant. Be concise and friendly."},
]

# Example REPL

## 3️⃣ Chat function with streaming

In [3]:
def chat_once(user_text: str) -> str:
    messages.append({"role":"user","content":user_text})
    inp = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt", reasoning_effort="low").to(device)

    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    gen_kwargs = dict(
        input_ids=inp,
        max_new_tokens=256,
        do_sample=True, top_p=0.9, temperature=0.7,
        eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
        use_cache=True, streamer=streamer,
    )
    t = threading.Thread(target=model.generate, kwargs=gen_kwargs); t.start()
    chunks=[]; print("Assistant: ", end="", flush=True)
    for tok in streamer: print(tok, end="", flush=True); chunks.append(tok)
    print()
    reply = "".join(chunks).strip()
    messages.append({"role":"assistant","content":reply})
    return reply


## 4️⃣ Interactive loop


In [4]:
while True:
    s = input("\nYou: ").strip()
    if s.lower() in {"bye","quit","exit","thanks","thank you, the problem has been resolved"}:
        print("Assistant: Glad I could help. 👋"); break
    chat_once(s)



You:  Hi there, how are you doing? I am not feeling well today 


Assistant: analysisNeed to respond empathetically.assistantfinalHi! I’m just a bunch of code, so I don’t have feelings, but I’m here to help. I’m sorry to hear you’re not feeling well. Is there anything I can do to help you feel better today?



You:  Well, I have a headache and some fever.


Assistant: analysisNeed to give advice, not medical diagnosis.assistantcommentary to=functions.run code{"name":"browser_search","arguments":"{\"query\":\"headache fever advice\"}"}commentarycommentary to=functions.runcommentary{"name":"browser_search","arguments":"{\"query\":\"headache fever advice\"}"}commentaryanalysisWe got results.assistantfinalI’m sorry to hear that—those symptoms can be pretty uncomfortable. While I’m not a medical professional, here are a few general suggestions that might help you feel a bit better:

| Symptom | Quick Self‑Care Tips |
|---------|----------------------|
| **Headache** | • Rest in a quiet, dimly lit room.<br>• Stay hydrated – sip water, herbal tea, or a clear broth.<br>• Try a cool compress on your forehead or neck. |
| **Fever** | • Take a lukewarm shower or bath; avoid hot water which can raise your core temperature.<br>• Wear loose, breathable clothing.<br>• Keep your room at a comfortable temperature (around 20–22 °C). |

### What to do next



You:  Great but what if I still do not feel well?


Assistant: analysisNeed to advise to see doctor.assistantfinalI’m sorry you’re still feeling unwell. If the headache and fever persist or worsen, it’s best to seek professional help. Here’s what you can do next:

1. **Contact a healthcare provider** – Call your doctor, nurse line, or go to an urgent‑care clinic if symptoms are severe (e.g., high fever >39 °C, persistent headache, difficulty breathing, confusion, or any new symptoms).

2. **Keep a symptom log** – Note the temperature, pain level, any other symptoms (nausea, rash, etc.). This information will be useful for the clinician.

3. **Follow any medical advice** – If you’re prescribed medication, take it as directed. If you’re told to rest, stay in a quiet, comfortable environment.

4. **Monitor for red‑flag signs** – If you develop:
   - Sudden severe headache (“thunderclap” headache)
   - Persistent vomiting or diarrhea
   - Difficulty breathing or chest pain
   - Severe or worsening fever
   – seek emergency care immediately.


You:  exit


Assistant: Glad I could help. 👋


## 🧮 Example Modifications

- Shorter / longer outputs: adjust max_new_tokens

- Deterministic: set do_sample=False

- Change assistant tone: modify the system message

- Reduce GPU memory usage: switch to 4-bit with load_in_4bit=True

## 🚀 Troubleshooting

| Problem              | Fix                                                       |
| -------------------- | --------------------------------------------------------- |
| `CUDA out of memory` | Use 4-bit quantization or reduce `max_seq_length`         |
| Model loads on CPU   | Verify GPU with `torch.cuda.is_available()`               |
| No streaming output  | Ensure you’re using `TextIteratorStreamer`                |
| Template error       | Remove `reasoning_effort` if unsupported in your template |


## 📜 Credits

- Models: Unsloth GPT-OSS family

- Libraries: unsloth, transformers, accelerate, bitsandbytes

- Author: Adapted for educational and research use 🧑‍💻